# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [14]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-97393615b3-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [3]:
# I'll try to retrieve the objs related to Woody Allen
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('obj', 'http://www.wikidata.org/entity/Q349357'), ('objName', 'Woody Herman')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('obj', 'http://www.wikidata.org/entity/Q1140914'), ('objName', 'Soon-Yi Previn')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('obj', 'http://www.wikidata.org/entity/Q22673707'), ('objName', 'Bechet Dumaine Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('obj', 'http://www.wikidata.org/entity/Q22673708'), ('objName', 'Manzie Tio Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('obj', 'http://www.wikidata.org/entity/Q18426'), ('objName', 'The Bronx')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('obj', 'http://www.wikidata.org/entity/Q102642'), ('objName', 'Diane Keaton')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('obj', 'http://www.wikidata.org/entity/Q3404523'), ('objName', 'David di Donatello for Best Foreign Actor')]
[('p', 'http://www.wikidata

In [4]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('obj', 'http://www.wikidata.org/entity/Q349357'), ('objName', 'Woody Herman')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q1140914'), ('objName', 'Soon-Yi Previn')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('obj', 'http://www.wikidata.org/entity/Q22673707'), ('objName', 'Bechet Dumaine Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('obj', 'http://www.wikidata.org/entity/Q22673708'), ('objName', 'Manzie Tio Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q18426'), ('objName', 'The Bronx')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('pName', 'unmarried partner'), ('obj', 'http://www.wikidata.org/entity/Q102642'), ('objName', 'Diane Keaton')]
[('p', 'http://www.wikidata.org/prop/dire

In [5]:
# I'll try to retrieve all the predicates and objs of Q971865 "What's Up, Tiger Lily?"
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pName', 'distributed by'), ('obj', 'http://www.wikidata.org/entity/Q465224'), ('objName', 'American International Pictures')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer'), ('obj', 'http://www.wikidata.org/entity/Q376138'), ('objName', 'Charles H. Joffe')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('pName', 'director of photography'), ('obj', 'http://www.wikidata.org/entity/Q6381680'), ('objName', 'Kazuo Yamada')]
[('p', 'http://www

Final query for this task

In [6]:
# write your final query
# I'll look at the instance of the film Q971865 "What's Up, Tiger Lily?"
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('objName', 'film')]
1


## Task 2
Identify the BGP for director

In [7]:
# write your queries
# I'll look at the director P57 of the film Q971865 "What's Up, Tiger Lily?"
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 wdt:P57 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('obj', 'http://www.wikidata.org/entity/Q27351'), ('objName', 'Senkichi Taniguchi')]
2


In [10]:
# I'll look at the predicate that link the film Q971865 to Q27351 Senkichi Taniguchi and Woody Allen Q25089
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q971865 ?p wd:Q27351 ;
        ?p wd:Q25089 .
   # get the label
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
1


In [15]:
# I understood that this is not the BGP that I should retrieve, because I have to retrieve the job 'director' instead of the action.
# To do so, I'll start by looking at the occupation Q2526255 film director that I've found on the third query of first task and I suppose to arrive at the node 'director' using P31 instance of
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2526255 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4220920'), ('objName', 'filmmaking occupation')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
2


In [16]:
# P31 instance of didn't work as expected, I'll try with P279 subclass of
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2526255 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
[('obj', 'http://www.wikidata.org/entity/Q3391743'), ('objName', 'visual artist')]
[('obj', 'http://www.wikidata.org/entity/Q3406858'), ('objName', 'film professional')]
3


Final query for this task

In [17]:
# P31 instance of didn't work as expected, I'll try with P279 subclass of
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2526255 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?objName.
   FILTER REGEX (?objName, "director")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
1


## Task 3
Identify the BGP for film genre

In [13]:
# write your queries
# I'll look at the the genre P136 obj of the film Q971865, that as retrieved before will be Q2297927 spy film, then look at its instance of where I expect to find the generic film genre 
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 wdt:P136 ?genre .
   ?genre wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('objName', 'film genre')]
1


Final query for this task

In [14]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 wdt:P136 ?genre .
   ?genre wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('objName', 'film genre')]
1


## Task 4
Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

IMPOSSIBILE TO HAVE A SOLUTION BECAUSE THE SNAPSHOT IS INCOMPLETE. THERE ISN'T ENOUGH INFORMATION ABOUT THE RELEASE OF FILMS IN COUNTRIES.

In [24]:
# write your queries
# I'll start by retrieving the western films looking at the subjects of 'subject instanceof film genres
queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q201658 .
   # get the label
   ?s sc:name ?sName.
   FILTER REGEX(?sName, "western")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q212781'), ('sName', 'spaghetti western')]
[('s', 'http://www.wikidata.org/entity/Q4836991'), ('sName', 'B western')]
[('s', 'http://www.wikidata.org/entity/Q83646243'), ('sName', 'western animation')]
[('s', 'http://www.wikidata.org/entity/Q78528852'), ('sName', 'neo-western')]
4


In [20]:
# I'm not a fan of western films, but I'm not sure that this is the correct genres I was looking for.
# I'll try to retrieve all the genres that belong to western
queryString = """
SELECT DISTINCT ?genre ?genreName
WHERE {
   # bind something
   ?s wdt:P136 ?genre .
   # get the label
   ?genre sc:name ?genreName
   FILTER REGEX(?genreName, ".*western.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q212781'), ('genreName', 'spaghetti western')]
[('genre', 'http://www.wikidata.org/entity/Q7988576'), ('genreName', 'television western')]
[('genre', 'http://www.wikidata.org/entity/Q4836991'), ('genreName', 'B western')]
[('genre', 'http://www.wikidata.org/entity/Q78528852'), ('genreName', 'neo-western')]
4


In [21]:
queryString = """
SELECT DISTINCT ?genre ?genreName
WHERE {
   # bind something
   ?s wdt:P136 ?genre .
   # get the label
   ?genre sc:name ?genreName
   FILTER REGEX(?genreName, ".*[Ww]estern.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q26263265'), ('genreName', 'Western art')]
[('genre', 'http://www.wikidata.org/entity/Q4235011'), ('genreName', 'Space Western')]
[('genre', 'http://www.wikidata.org/entity/Q9730'), ('genreName', 'Western classical music')]
[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genreName', 'Western film')]
[('genre', 'http://www.wikidata.org/entity/Q491158'), ('genreName', 'Revisionist Western')]
[('genre', 'http://www.wikidata.org/entity/Q12197074'), ('genreName', 'Western novel')]
[('genre', 'http://www.wikidata.org/entity/Q367591'), ('genreName', 'Western literature')]
[('genre', 'http://www.wikidata.org/entity/Q7439349'), ('genreName', 'Scrumpy and Western')]
[('genre', 'http://www.wikidata.org/entity/Q212781'), ('genreName', 'spaghetti western')]
[('genre', 'http://www.wikidata.org/entity/Q21590660'), ('genreName', 'Western')]
[('genre', 'http://www.wikidata.org/entity/Q7988576'), ('genreName', 'television western')]
[('genre', '

In [22]:
# I'll try to see if there are specific film genre that I didn't found before because of the wrong regex
queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q201658 .
   # get the label
   ?s sc:name ?sName.
   FILTER REGEX(?sName, ".*[Ww]estern.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q268854'), ('sName', 'Red Western')]
[('s', 'http://www.wikidata.org/entity/Q4235011'), ('sName', 'Space Western')]
[('s', 'http://www.wikidata.org/entity/Q172980'), ('sName', 'Western film')]
[('s', 'http://www.wikidata.org/entity/Q491158'), ('sName', 'Revisionist Western')]
[('s', 'http://www.wikidata.org/entity/Q212781'), ('sName', 'spaghetti western')]
[('s', 'http://www.wikidata.org/entity/Q4836991'), ('sName', 'B western')]
[('s', 'http://www.wikidata.org/entity/Q6804274'), ('sName', 'meat pie Western')]
[('s', 'http://www.wikidata.org/entity/Q5461696'), ('sName', 'Florida Western')]
[('s', 'http://www.wikidata.org/entity/Q4674071'), ('sName', 'Acid Western')]
[('s', 'http://www.wikidata.org/entity/Q83646243'), ('sName', 'western animation')]
[('s', 'http://www.wikidata.org/entity/Q78528852'), ('sName', 'neo-western')]
[('s', 'http://www.wikidata.org/entity/Q8066387'), ('sName', 'Zapata Western')]
12


In [23]:
# Now that I have found the correct film genre Q172980 western film, I'll retrieve the films that belong to this genre
queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
       wdt:P136 wd:Q172980 .
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q15882802'), ('filmName', 'The Storm')]
[('film', 'http://www.wikidata.org/entity/Q16200305'), ('filmName', 'Fangs of the Arctic')]
[('film', 'http://www.wikidata.org/entity/Q17011616'), ('filmName', 'Mr. Texas')]
[('film', 'http://www.wikidata.org/entity/Q17036710'), ('filmName', 'The Drifting Avenger')]
[('film', 'http://www.wikidata.org/entity/Q17052173'), ('filmName', 'The Desert of the Lost')]
[('film', 'http://www.wikidata.org/entity/Q17053125'), ('filmName', 'Between Dangers')]
[('film', 'http://www.wikidata.org/entity/Q7569044'), ('filmName', 'South of Santa Fe')]
[('film', 'http://www.wikidata.org/entity/Q17184091'), ('filmName', 'With Buffalo Bill on the U. P. Trail')]
[('film', 'http://www.wikidata.org/entity/Q17190227'), ('filmName', 'Easy Going')]
[('film', 'http://www.wikidata.org/entity/Q17639337'), ('filmName', 'Skedaddle Gold')]
[('film', 'http://www.wikidata.org/entity/Q17639444'), ('filmName', 'Speedy Spurs')]
[('film

In [25]:
# Now I'll try to see if there is a predicate that link a film with Italy Q38 (from previous notebook), in order to understand how film relase are stored.
queryString = """
SELECT DISTINCT ?film ?filmName ?p ?pName
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
       wdt:P136 wd:Q172980 ;
       ?p wd:Q38.
   # get the label
   ?film sc:name ?filmName.
   ?p sc:name ?pName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q829365'), ('filmName', "Don't Touch the White Woman!"), ('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('film', 'http://www.wikidata.org/entity/Q521876'), ('filmName', 'Last of the Renegades'), ('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('film', 'http://www.wikidata.org/entity/Q535427'), ('filmName', 'The Desperado Trail'), ('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('film', 'http://www.wikidata.org/entity/Q153677'), ('filmName', 'For a Few Dollars More'), ('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('film', 'http://www.wikidata.org/entity/Q3823650'), ('filmName', 'Prey of Vultures'), ('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('film', 'http://www.wikidata.org/entity/Q1196578'), ('filmName', 'Cuatro balazos'), ('p', 'http://www.wikidata.or

In [26]:
# Since there's no predicate that link the film to its release in Italy, I'll look more genericaly if there's a release predicate
queryString = """
SELECT DISTINCT ?film ?filmName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
       wdt:P136 wd:Q172980 ;
       ?p ?obj.
   # get the label
   ?film sc:name ?filmName .
   ?p sc:name ?pName .
   ?obj sc:name ?objName
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmName', 'The Hateful Eight'), ('p', 'http://www.wikidata.org/prop/direct/P5201'), ('pName', 'IMDA rating'), ('obj', 'http://www.wikidata.org/entity/Q52382317'), ('objName', 'R21')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmName', 'The Hateful Eight'), ('p', 'http://www.wikidata.org/prop/direct/P4437'), ('pName', 'FPB rating'), ('obj', 'http://www.wikidata.org/entity/Q42012509'), ('objName', '18')]
[('film', 'http://www.wikidata.org/entity/Q17184091'), ('filmName', 'With Buffalo Bill on the U. P. Trail'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor'), ('obj', 'http://www.wikidata.org/entity/Q4704391'), ('objName', 'Al Martin')]
[('film', 'http://www.wikidata.org/entity/Q7911233'), ('filmName', 'Valerie'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor'), ('obj', 'http://www.wikidata.org/entity/Q1173828'), ('objName', 'David Bretherton')]
[('fi

In [28]:
# For a better interpretation of the results, I'll look at the predicates and objs of the film Q18225084
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q18225084 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q26209411'), ('objName', 'Son of a Gun')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q26209418'), ('objName', "Domergue's Got a Secret")]
[('p', 'http://www.wikidata.org/prop/direct/P9215'), ('pName', 'set during recurring event'), ('obj', 'http://www.wikidata.org/entity/Q1311'), ('objName', 'winter')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q3772'), ('objName', 'Quentin Tarantino')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('obj', 'http://www.wikidata.org/entity/Q3772'), ('objName', 'Quentin Tarantino')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter'), ('obj', 'http://www.wikidata.org/entity/Q3772'), ('objName', 'Quentin Tarantino')]
[('p', 'http://www.wik

In [30]:
# Still nothing, can't find a release property. I'll try to look at the objs related to film
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q1660187'), ('objName', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P345'), ('objName', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1040'), ('objName', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P344'), ('objName', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pName', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q17738'), ('objName', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q55321'), ('o

In [31]:
# Still nothing, I'll try to look at the subjects that has film as related obj
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6171351'), ('sName', 'Template:Infobox film'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q79704281'), ('sName', 'Template:Disney+ movie'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q30092790'), ('sName', 'Christine Wiegand'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q250608'), ('sName', 'Ali Hatami'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q189526'), ('sName', 'Brian De Palma'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q524370'), ('sName', 'Joyce Wieland'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http:/

In [33]:
# I'll try to filter out all the field of work relations
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER NOT EXISTS {?s wdt:P101 wd:Q11424 }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6171351'), ('sName', 'Template:Infobox film'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q79704281'), ('sName', 'Template:Disney+ movie'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q1124579'), ('sName', 'Condor Films'), ('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]
[('s', 'http://www.wikidata.org/entity/Q16539202'), ('sName', 'Champion Film Company'), ('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]
[('s', 'http://www.wikidata.org/entity/Q17088846'), ('sName', 'Spring Films'), ('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]
[('s', 'http://www.wikidata.org/entity/Q18155354'), ('sName', 'TSG Entertainment'), ('p', 'http://www.wikidata.org/p

In [34]:
# I'll try to filter out all the field of work relations
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER NOT EXISTS {
       ?s wdt:P101 wd:Q11424 .
       ?s wdt:P31 wd:Q11424 .
       ?s wdt:P1056 wd:Q11424 .
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6171351'), ('sName', 'Template:Infobox film'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q79704281'), ('sName', 'Template:Disney+ movie'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q30092790'), ('sName', 'Christine Wiegand'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q250608'), ('sName', 'Ali Hatami'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q189526'), ('sName', 'Brian De Palma'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q524370'), ('sName', 'Joyce Wieland'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http:/

In [3]:
# I'll try to filter with a regex on release
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?s, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
# I'll try to filter with a regex on release
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?p, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# I'll try to filter with a regex on release
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?sName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q29022287'), ('sName', 'Day Release'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1762957'), ('sName', 'Release the Prisoners to Spring'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1823809'), ('sName', 'Catch and Release'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q27922003'), ('sName', 'Conditional release'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q17009177'), ('sName', 'Magdalena: Released from Shame'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q7310915'), ('sName', 'Release'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wik

In [6]:
# I'll try to filter with a regex on release
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q11424 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?pName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
# I'll try to filter with a regex on release
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
# For a better interpretation of the results, I'll look at the predicates and objs of the film Q18225084 with a filter regex on release
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q18225084 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
   FILTER REGEX (?objName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [9]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q18225084 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?sName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [3]:
# I'll try to reach a release node with the film Q971865
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q971865 .
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER REGEX (?sName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
   FILTER REGEX (?objName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
   FILTER REGEX (?pName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# I'll try to look at the predicates and objs list of movies released by New Line Cinema Q2072552
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q2072552 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
   FILTER REGEX (?pName, ".*[Rr]elease.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
# I'll try to look at the predicates and objs list of movies released by New Line Cinema Q2072552
queryString = """
SELECT DISTINCT?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q2072552 ?p ?obj.
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('obj', 'http://www.wikidata.org/entity/Q11424'), ('objName', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13406463'), ('objName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('obj', 'http://www.wikidata.org/entity/Q6841017'), ('objName', 'Category:New Line Cinema films')]
3


In [8]:
#I'll try to see the films that belong to that list Q2072552
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q2072552 .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6841017'), ('sName', 'Category:New Line Cinema films'), ('p', 'http://www.wikidata.org/prop/direct/P1753'), ('pName', 'list related to category')]
1


In [10]:
# I'll try another way to reach a node "film release"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?sName, "film release")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


In [2]:
# I'll try another way to reach a node "film release"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "film release")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35574063'), ('sName', 'Physicochemical regulation of biofilm formation.'), ('p', 'http://www.wikidata.org/prop/direct/P2860'), ('pName', 'cites work'), ('obj', 'http://www.wikidata.org/entity/Q33984665'), ('objName', 'Surface-grafted, environmentally sensitive polymers for biofilm release.')]
[('s', 'http://www.wikidata.org/entity/Q33987328'), ('sName', 'Use of self-assembled monolayers of different wettabilities to study surface selection and primary adhesion processes of green algal (Enteromorpha) zoospores.'), ('p', 'http://www.wikidata.org/prop/direct/P2860'), ('pName', 'cites work'), ('obj', 'http://www.wikidata.org/entity/Q33984665'), ('objName', 'Surface-grafted, environmentally sensitive polymers for biofilm release.')]
[('s', 'http://www.wikidata.org/entity/Q41031539'), ('sName', 'Effect of substratum surface chemistry and surface energy on attachment of marine bacteria and algal spores'), ('p', 'http://www.wikidata.org/prop/dire

In [3]:
# Finally, I have found the node film release Q5449034. I'll look at its predicates and objs
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q5449034 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q204854'), ('objName', 'première')]
1


In [4]:
# I'll look at the subjects that have Q5449034 as obj
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q5449034 .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3491297'), ('sName', 'limited release'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q28136699'), ('sName', 'The Dog'), ('p', 'http://www.wikidata.org/prop/direct/P793'), ('pName', 'significant event')]
[('s', 'http://www.wikidata.org/entity/Q4019680'), ('sName', 'wide release'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
3


In [5]:
# Since there's only one film that is linked to the node "film release", I'll try to retrieve the films linked to node Q4019680 wide release
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q4019680 .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q4019680 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q5449034'), ('objName', 'film release')]
1


In [7]:
# As we can see, there's no films linked to node wide release Q4019680.
# Before giving upI'll try to retrieve the nodes that contains the word "release"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "[Rr]elease")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8151671'), ('sName', 'Category:1962 compilation albums'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('obj', 'http://www.wikidata.org/entity/Q105701834'), ('objName', 'compilation album released this year')]
[('s', 'http://www.wikidata.org/entity/Q8153381'), ('sName', 'Category:1965 compilation albums'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('obj', 'http://www.wikidata.org/entity/Q105701834'), ('objName', 'compilation album released this year')]
[('s', 'http://www.wikidata.org/entity/Q8168187'), ('sName', 'Category:1986 compilation albums'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('obj', 'http://www.wikidata.org/entity/Q105701834'), ('objName', 'compilation album released this year')]
[('s', 'http://www.wikidata.org/entity/Q8168637'), ('sName', 'Category:Warner Music Group compilation album

In [8]:
# I'll search for the nodes that contains the exact word "release"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "^[Rr]elease$")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4790184'), ('sName', 'Ariadne auf Naxos discography'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('obj', 'http://www.wikidata.org/entity/Q2031291'), ('objName', 'release')]
[('s', 'http://www.wikidata.org/entity/Q51717728'), ('sName', 'Métamorphose'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q2031291'), ('objName', 'release')]
[('s', 'http://www.wikidata.org/entity/Q18208934'), ('sName', 'Les Huguenots discography'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('obj', 'http://www.wikidata.org/entity/Q2031291'), ('objName', 'release')]
[('s', 'http://www.wikidata.org/entity/Q273057'), ('sName', 'discography'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q2031291'), ('objName', 'release')]
[('s', 'http://www.wikida

In [10]:
# Since previous query retrieved only songs, I'll try to match only film Q11424
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424;
       ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "^[Rr]elease$")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [11]:
# I'll try to filter the prepositions on "^[Rr]elease$"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424;
       ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?pName, "^[Rr]elease$")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [12]:
# I'll try to filter the prepositions on "^[Rr]elease$"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424;
       ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?pName, ".*[Ff]ilm [Rr]elease.*")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [13]:
# I'll try to filter the prepositions on "^[Rr]elease$"
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424;
       ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER REGEX (?pName, ".*[Rr]elease.*")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q377923'), ('sName', 'Some Came Running'), ('p', 'http://www.wikidata.org/prop/direct/P406'), ('pName', 'soundtrack release'), ('obj', 'http://www.wikidata.org/entity/Q63524337'), ('objName', 'Some Came Running – Original Film Score')]
[('s', 'http://www.wikidata.org/entity/Q377923'), ('sName', 'Some Came Running'), ('p', 'http://www.wikidata.org/prop/direct/P406'), ('pName', 'soundtrack release'), ('obj', 'http://www.wikidata.org/entity/Q96576549'), ('objName', 'Song from "Some Came Running" (To Love and Be Loved) / No One Ever Tells You')]
[('s', 'http://www.wikidata.org/entity/Q267721'), ('sName', 'Grease'), ('p', 'http://www.wikidata.org/prop/direct/P406'), ('pName', 'soundtrack release'), ('obj', 'http://www.wikidata.org/entity/Q2698545'), ('objName', 'Grease – The Original Soundtrack from the Motion Picture')]
[('s', 'http://www.wikidata.org/entity/Q375855'), ('sName', 'American Graffiti'), ('p', 'http://www.wikidata.org/prop/direct/

In [19]:
# I'll try to filter out the soundtracks
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424;
       ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   ?obj sc:name ?objName .
   FILTER NOT EXISTS {?s wdt:P406 ?obj.}
   FILTER REGEX (?pName, ".*[Rr]elease.*")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [24]:
# I'll try by using property P577 publication date of film
queryString = """
SELECT DISTINCT ?film ?filmName ?date
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424;
       wdt:P577 ?date .
   # get the label
   ?film sc:name ?filmName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q8178307'), ('filmName', '15 Park Avenue'), ('date', '2005-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q8197668'), ('filmName', 'Idiot Love'), ('date', '2004-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q319263'), ('filmName', 'Doctor in the House'), ('date', '1954-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('filmName', "Sunday League - Pepik Hnatek's Final Match"), ('date', '2012-03-29T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('filmName', 'Ra.One'), ('date', '2011-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('filmName', 'Ra.One'), ('date', '2011-10-26T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q13637926'), ('filmName', 'Indra'), ('date', '2002-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q13638153'), ('filmName', 'Inner Senses'), ('date', '2002-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.

In [25]:
# I'll try by using property P577 publication date of film that hase genre western
queryString = """
SELECT DISTINCT ?film ?filmName ?date
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424;
       wdt:P136 wd:Q172980 ;
       wdt:P577 ?date .
   # get the label
   ?film sc:name ?filmName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)
       

Results
[('film', 'http://www.wikidata.org/entity/Q15882802'), ('filmName', 'The Storm'), ('date', '1930-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q16200305'), ('filmName', 'Fangs of the Arctic'), ('date', '1953-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17011616'), ('filmName', 'Mr. Texas'), ('date', '1951-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17036710'), ('filmName', 'The Drifting Avenger'), ('date', '1968-06-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17052173'), ('filmName', 'The Desert of the Lost'), ('date', '1927-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17053125'), ('filmName', 'Between Dangers'), ('date', '1927-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q7569044'), ('filmName', 'South of Santa Fe'), ('date', '1942-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17184091'), ('filmName', 'With Buffalo Bill on the U. P. Trail'), ('date', '1926-03-01T00:

In [28]:
# OK, I think that finally I have found the way.
# I'll see if date has a label, even if I don't think so
queryString = """
SELECT DISTINCT ?film ?filmName ?date
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424;
       wdt:P136 wd:Q172980 ;
       wdt:P577 ?date .
   # get the label
   ?film sc:name ?filmName .
   ?date sc:name ?dateName
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [ ]:
# write your final query

# IMPOSSIBILE TO HAVE A SOLUTION BECAUSE THE SNAPSHOT IS INCOMPLETE. THERE ISN'T ENOUGH INFORMATION ABOUT THE RELEASE OF FILMS IN A SPECIFIC COUNTRY

## Task 5
Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films).

In [20]:
# write your queries
# I'll start by looking at the films directed by Woody Allen Q25089
queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089 .
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?")]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('filmName', 'Broadway Danny Rose')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('filmName', 'Match Point')]
[('film', 'http://www.wikidata.org/entity/Q740143'), ('filmName', 'Husbands and Wives')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('filmName', 'Bananas')]
[('film', 'http://www.wikidata.org/entity/Q1321004'), ('filmName', 'You Will Meet a Tall Dark Stranger')]
[('film'

In [22]:
# for the film Q42307699 I'll retrieve its predicates and objs
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q42307699 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q160432'), ('objName', 'Jude Law')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q228943'), ('objName', 'Elle Fanning')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer'), ('obj', 'http://www.wikidata.org/entity/Q3831197'), ('objName', 'Letty Aronson')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor'), ('obj', 'http://www.wikidata.org/entity/Q4726932'), ('objName', 'Alisa Lepselter')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), 

In [29]:
# I'll try by using P577 publication date (previously retrieved on task 4) on this film, since there's no references to any pubblication on these results
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?obj .
   # get the label
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [30]:
# I'll try without matching the label
queryString = """
SELECT DISTINCT ?obj
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?obj .
   # get the label
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '2019-07-26T00:00:00Z')]
[('obj', '2019-09-18T00:00:00Z')]
[('obj', '2019-12-05T00:00:00Z')]
[('obj', '2019-12-12T00:00:00Z')]
4


In [32]:
# OK, there's four publication dates that for sure refers to publication in different country.
# I'll try to retrieve the predicates and objs of the dates
queryString = """
SELECT DISTINCT ?date ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   ?date ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [37]:
# I'll try to retrieve the predicates and objs of the dates
queryString = """
SELECT DISTINCT ?date ?p ?pName ?obj
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   ?date ?p ?obj .
   # get the label
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [39]:
# I'll try to retrieve the predicates and objs of the dates without matching their labels
queryString = """
SELECT DISTINCT ?date ?p ?obj
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   ?date ?p ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [40]:
# for the film Q42307699 I'll retrieve its predicates and objs without matching the label on obj
queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   # bind something
   wd:Q42307699 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor'), ('obj', 'http://www.wikidata.org/entity/Q4726932')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID'), ('obj', '258065')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q1054574')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q157443')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title'), ('obj', 'A Rainy Day in New York')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q27847754')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q75837423')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q160432')]
[('p', 'h

In [2]:
# I was trying to understand if there's a publication country associated with the publication date in order to complete the task 4, but there isn't.
# Now I'll try to retrieve the year for the publications of this film Q42307699

queryString = """
SELECT DISTINCT (YEAR(?obj) as ?year)
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?obj .
   # get the label
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '2019')]
1


In [4]:
# trying to get only the first date of publication
queryString = """
SELECT DISTINCT ?date
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   FILTER NOT EXISTS {
       wd:Q42307699 wdt:P577 ?date2
       FILTER (?date2 > ?date)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('date', '2019-12-12T00:00:00Z')]
1


In [5]:
# trying to get only the first date of publication
queryString = """
SELECT DISTINCT ?date
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   FILTER NOT EXISTS {
       wd:Q42307699 wdt:P577 ?date2
       FILTER (?date > ?date2)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('date', '2019-07-26T00:00:00Z')]
1


In [6]:
# I'll try to get the year of the first date of publication, just to be sure
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?year)
WHERE {
   # bind something
   wd:Q42307699 wdt:P577 ?date .
   FILTER NOT EXISTS {
       wd:Q42307699 wdt:P577 ?date2
       FILTER (?date > ?date2)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '2019')]
1


In [ ]:
# Now I'll take some films directed by Q25089 Woody Allen and get their publication date.
queryString = """
SELECT DISTINCT ?film ?filmName (YEAR(?date) AS ?year)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089 ;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       wd:Q42307699 wdt:P577 ?date2
       FILTER (?date > ?date2)
   }
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('year', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('year', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('year', '1990')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('year', '1991')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?"), ('year', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('year', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('year', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('year', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('year', '1996')]
[('film', 'http://www.wikidata.org

In [8]:
# On previous query I was wrongly matching the dates on the film Q42307699
queryString = """
SELECT DISTINCT ?film ?filmName (YEAR(?date) AS ?year)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089 ;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2
       FILTER (?date > ?date2)
   }
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('year', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('year', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('year', '1990')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?"), ('year', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('year', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('year', '1995')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('filmName', 'Broadway Danny Rose'), ('year', '1984')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('filmName', 'Match Point'), ('year', '2005')]
[('film', 'http://www.wikidata.org/entity/Q740143'), ('filmName', 'Husbands and Wives'), ('year', '1992')]
[('film', 'http://www.wikidata

In [14]:
# Now i'll group by year
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089 ;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '2001'), ('numFilms', '2')]
[('year', '1995'), ('numFilms', '1')]
[('year', '1984'), ('numFilms', '1')]
[('year', '1989'), ('numFilms', '2')]
[('year', '2015'), ('numFilms', '1')]
[('year', '1996'), ('numFilms', '1')]
[('year', '1978'), ('numFilms', '1')]
[('year', '1966'), ('numFilms', '1')]
[('year', '2014'), ('numFilms', '1')]
[('year', '2020'), ('numFilms', '1')]
[('year', '2013'), ('numFilms', '1')]
[('year', '2019'), ('numFilms', '1')]
[('year', '1986'), ('numFilms', '1')]
[('year', '2003'), ('numFilms', '1')]
[('year', '1980'), ('numFilms', '1')]
[('year', '1979'), ('numFilms', '1')]
[('year', '2000'), ('numFilms', '1')]
[('year', '1972'), ('numFilms', '1')]
[('year', '2002'), ('numFilms', '1')]
[('year', '2011'), ('numFilms', '1')]
[('year', '1990'), ('numFilms', '1')]
[('year', '1994'), ('numFilms', '1')]
[('year', '2005'), ('numFilms', '1')]
[('year', '1992'), ('numFilms', '1')]
[('year', '2010'), ('numFilms', '1')]
[('year', '1998'), ('numFilms', '1')]
[('y

In [15]:
# I'll order by year for a better interpretation of the results
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089 ;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '1966'), ('numFilms', '1')]
[('year', '1969'), ('numFilms', '1')]
[('year', '1971'), ('numFilms', '1')]
[('year', '1972'), ('numFilms', '1')]
[('year', '1973'), ('numFilms', '1')]
[('year', '1975'), ('numFilms', '1')]
[('year', '1977'), ('numFilms', '1')]
[('year', '1978'), ('numFilms', '1')]
[('year', '1979'), ('numFilms', '1')]
[('year', '1980'), ('numFilms', '1')]
[('year', '1982'), ('numFilms', '1')]
[('year', '1983'), ('numFilms', '1')]
[('year', '1984'), ('numFilms', '1')]
[('year', '1985'), ('numFilms', '1')]
[('year', '1986'), ('numFilms', '1')]
[('year', '1987'), ('numFilms', '2')]
[('year', '1988'), ('numFilms', '1')]
[('year', '1989'), ('numFilms', '2')]
[('year', '1990'), ('numFilms', '1')]
[('year', '1991'), ('numFilms', '1')]
[('year', '1992'), ('numFilms', '1')]
[('year', '1993'), ('numFilms', '1')]
[('year', '1994'), ('numFilms', '1')]
[('year', '1995'), ('numFilms', '1')]
[('year', '1996'), ('numFilms', '1')]
[('year', '1997'), ('numFilms', '1')]
[('y

In [19]:
# I'll order by year for a better interpretation of the results
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER EXISTS{
       {
           ?film wdt:P57 wd:Q25089.
        }
        UNION
        {
           ?film wdt:P57 wd:Q3772 .
        }
   }
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '1966'), ('numFilms', '1')]
[('year', '1969'), ('numFilms', '1')]
[('year', '1971'), ('numFilms', '1')]
[('year', '1972'), ('numFilms', '1')]
[('year', '1973'), ('numFilms', '1')]
[('year', '1975'), ('numFilms', '1')]
[('year', '1977'), ('numFilms', '1')]
[('year', '1978'), ('numFilms', '1')]
[('year', '1979'), ('numFilms', '1')]
[('year', '1980'), ('numFilms', '1')]
[('year', '1982'), ('numFilms', '1')]
[('year', '1983'), ('numFilms', '1')]
[('year', '1984'), ('numFilms', '1')]
[('year', '1985'), ('numFilms', '1')]
[('year', '1986'), ('numFilms', '1')]
[('year', '1987'), ('numFilms', '2')]
[('year', '1988'), ('numFilms', '1')]
[('year', '1989'), ('numFilms', '2')]
[('year', '1989'), ('numFilms', '1')]
[('year', '1990'), ('numFilms', '1')]
[('year', '1991'), ('numFilms', '1')]
[('year', '1992'), ('numFilms', '1')]
[('year', '1993'), ('numFilms', '1')]
[('year', '1994'), ('numFilms', '1')]
[('year', '1995'), ('numFilms', '1')]
[('year', '1996'), ('numFilms', '1')]
[('y

In [20]:
# I'll order by year for a better interpretation of the results
queryString = """
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER EXISTS{
       {
           ?film wdt:P57 wd:Q25089 .
        }
        UNION
        {
           ?film wdt:P57 wd:Q3772 .
        }
   }
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1966'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q27351'), ('directorName', 'Senkichi Taniguchi'), ('year', '1966'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1969'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1971'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1972'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1973'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1975'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1977

In [21]:
# I'll order by year for a better interpretation of the results
queryString = """
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1966'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1969'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1971'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1972'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1973'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1975'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1977'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1978'), ('n

In [23]:
# I'll group by the director on a 
queryString = """
SELECT DISTINCT ?director ?directorName ?year (MAX(?numFilms) AS ?maxFilms)
WHERE
    {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
    }
GROUP BY ?year ?numFilms ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1994'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1990'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2007'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1972'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2015'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2006'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1977'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1995'), ('m

In [28]:
# I was wrongly grouping by the year
queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?numFilms) AS ?maxFilms)
WHERE
    {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
    }
GROUP BY ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxFilms', '2')]
2


In [4]:
queryString = """
SELECT DISTINCT ?director ?directorName ?year (MAX(?numFilms) AS ?maxFilms) 
WHERE
    {
    {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
    }
    }
GROUP BY ?director ?directorName ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1998'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('year', '2003'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1990'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2015'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2002'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('year', '2019'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1994'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1

In [7]:
# I'll try another way to do it
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '1966'), ('numFilms', '1')]
[('year', '1969'), ('numFilms', '1')]
[('year', '1971'), ('numFilms', '1')]
[('year', '1972'), ('numFilms', '1')]
[('year', '1973'), ('numFilms', '1')]
[('year', '1975'), ('numFilms', '1')]
[('year', '1977'), ('numFilms', '1')]
[('year', '1978'), ('numFilms', '1')]
[('year', '1979'), ('numFilms', '1')]
[('year', '1980'), ('numFilms', '1')]
[('year', '1982'), ('numFilms', '1')]
[('year', '1983'), ('numFilms', '1')]
[('year', '1984'), ('numFilms', '1')]
[('year', '1985'), ('numFilms', '1')]
[('year', '1986'), ('numFilms', '1')]
[('year', '1987'), ('numFilms', '2')]
[('year', '1988'), ('numFilms', '1')]
[('year', '1989'), ('numFilms', '2')]
[('year', '1990'), ('numFilms', '1')]
[('year', '1991'), ('numFilms', '1')]
[('year', '1992'), ('numFilms', '1')]
[('year', '1993'), ('numFilms', '1')]
[('year', '1994'), ('numFilms', '1')]
[('year', '1995'), ('numFilms', '1')]
[('year', '1996'), ('numFilms', '1')]
[('year', '1997'), ('numFilms', '1')]
[('y

In [9]:
queryString = """
SELECT DISTINCT ?year (MAX(?numFilms) AS ?maxFilms)
WHERE{
SELECT DISTINCT (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 wd:Q25089;
        wdt:P577 ?date .
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date
}
GROUP BY ?year
ORDER BY ?year
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('year', '1966'), ('maxFilms', '1')]
[('year', '1969'), ('maxFilms', '1')]
[('year', '1971'), ('maxFilms', '1')]
[('year', '1972'), ('maxFilms', '1')]
[('year', '1973'), ('maxFilms', '1')]
[('year', '1975'), ('maxFilms', '1')]
[('year', '1977'), ('maxFilms', '1')]
[('year', '1978'), ('maxFilms', '1')]
[('year', '1979'), ('maxFilms', '1')]
[('year', '1980'), ('maxFilms', '1')]
[('year', '1982'), ('maxFilms', '1')]
[('year', '1983'), ('maxFilms', '1')]
[('year', '1984'), ('maxFilms', '1')]
[('year', '1985'), ('maxFilms', '1')]
[('year', '1986'), ('maxFilms', '1')]
[('year', '1987'), ('maxFilms', '2')]
[('year', '1988'), ('maxFilms', '1')]
[('year', '1989'), ('maxFilms', '2')]
[('year', '1990'), ('maxFilms', '1')]
[('year', '1991'), ('maxFilms', '1')]
[('year', '1992'), ('maxFilms', '1')]
[('year', '1993'), ('maxFilms', '1')]
[('year', '1994'), ('maxFilms', '1')]
[('year', '1995'), ('maxFilms', '1')]
[('year', '1996'), ('maxFilms', '1')]
[('year', '1997'), ('maxFilms', '1')]
[('y

In [14]:
# In order to have the year on the results, I'll have to add the year on the group by, but this will retrieve all the years as in the previous query.
# I have to group by the director, so I can have the maxFilms. Then, I'll find a way to get the year.

queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?numFilms) AS ?maxFilms)
WHERE
    {
        {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
        }
    }
GROUP BY ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxFilms', '2')]
2


In [18]:
# I'll try to get the year
# My idea is to match the last query with the query that retrieve numFilms year by year.
queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?numFilms) AS ?maxFilms)
WHERE
    {
        {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
        }
        FILTER(?maxFilms=?numFilms)
    }
GROUP BY ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [24]:
# I'll try to add a subquery
queryString = """
SELECT DISTINCT ?director ?directorName ?year2 ?maxFilms
WHERE
{
    {
    SELECT DISTINCT ?director ?directorName (MAX(?numFilms) AS ?maxFilms)
    WHERE
    {
        {
        SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
        WHERE {
           # bind something
           ?film wdt:P31 wd:Q11424 ;
                wdt:P57 ?director;
                wdt:P577 ?date .
           ?director sc:name ?directorName .
           FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
           FILTER NOT EXISTS {
               ?film wdt:P577 ?date2 .
               FILTER (?date > ?date2)
           }
        }
        GROUP BY ?year ?date ?director ?directorName
        }
    }
    GROUP BY ?director ?directorName
    }
    FILTER(?maxFilms = ?numFilms)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [12]:
# I'll try a different way.
# I'll retrieve the num of films for each year of Woody Allen, then order by numFilms ASC
queryString = """
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q25089)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1989'), ('numFilms', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1987'), ('numFilms', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2001'), ('numFilms', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2014'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1975'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1993'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1969'), ('numFilms', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '2009'), ('n

In [18]:
# I'll retrieve the num of films for each year of Woody Allen, then order by numFilms and take only the first row
queryString = """
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q25089)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1989'), ('numFilms', '2')]
1


In [29]:
# Now I'll retrieve the num of films for each year of Woody Allen and for Quentin Tarantino in a separated query, then order by numFilms and take only the first row
queryString = """
SELECT DISTINCT ?director ?directorName ?year ?numFilms
WHERE{
{
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q25089)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 1
}
UNION
{
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q3772)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 1
}
}

"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1989'), ('numFilms', '2')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('year', '2005'), ('numFilms', '1')]
2


Final query for this task

In [30]:
# write your final query
# Now I'll retrieve the num of films for each year of Woody Allen and for Quentin Tarantino in two separated query, then order by numFilms and take only the first row. Then I'll use UNION to concatenate the two queries
queryString = """
SELECT DISTINCT ?director ?directorName ?year ?numFilms
WHERE{
{
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q25089)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 1
}
UNION
{
SELECT DISTINCT ?director ?directorName (YEAR(?date) AS ?year) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P577 ?date .
   ?director sc:name ?directorName .
   FILTER(?director=wd:Q3772)
   FILTER NOT EXISTS {
       ?film wdt:P577 ?date2 .
       FILTER (?date > ?date2)
   }
}
GROUP BY ?year ?date ?director ?directorName
ORDER BY DESC(?numFilms)
LIMIT 1
}
}

"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('year', '1989'), ('numFilms', '2')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('year', '2005'), ('numFilms', '1')]
2


## Task 6
Identify the BGP for Academy Awards

In [5]:
# write your queries
queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   # bind something
   ?film wdt:P31 wd:Q11424 ;
        wdt:P166 ?award .
    ?award sc:name ?awardName .
    FILTER (?awardName = "Academy Awards")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
#I'll look at the predicate "instance of" of Q488651 Academy Award for Best Original Score
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('objName', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q96474685'), ('objName', 'award for best original music')]
2


In [11]:
#I'll look at the predicate "instance of" of Q488651 Academy Award for Best Original Score
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "Academy Awards")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('objName', 'Academy Awards')]
1


Final query for this task

In [12]:
# write your final query
#I'll look at the predicate "instance of" of Q488651 Academy Award for Best Original Score
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   ?obj sc:name ?objName .
   FILTER REGEX (?objName, "Academy Awards")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('objName', 'Academy Awards')]
1


## Task 7
Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

In [17]:
# write your queries
# I'll retrieve the academy awards won by the films of the two directors
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName ?academyAward ?academyAwardName
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('director', 'http://www.wikidata.org/entity/Q3772

In [18]:
# I'll group by film and director
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('numAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmName', 'Hannah and Her Sisters'), ('numAwards', '3')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('numAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('numAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmName', 'Django Unchained

In [23]:
# I'll group by film
queryString = """
SELECT DISTINCT ?film ?filmName (MAX(?numAwards) AS ?maxAwards)
WHERE {
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
GROUP BY ?film ?filmName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('maxAwards', '2')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q2907178'), ('filmName', 'Blue Jasmine'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q212123'), ('filmName', 'Vicky Cristina Barcelona'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmName', 'The Hateful Eight'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q845057'), ('filmName', 'Hannah and Her Sisters'), ('maxAwards', '3')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('maxAwards', '1')]
[('film', 'http://www.wikidata.org/entity/Q571032'), ('filmName', 'Django Unchained'), ('maxAwards', '2')]

In [24]:
# I'll group by director
queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?numAwards) AS ?maxAwards)
WHERE {
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
GROUP BY ?director ?directorName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxAwards', '4')]
2


In [25]:
# I'll group by director and film
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (MAX(?numAwards) AS ?maxAwards)
WHERE {
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
GROUP BY ?director ?directorName ?film ?filmName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmName', 'Django Unchained'), ('maxAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q2907178'), ('filmName', 'Blue Jasmine'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('maxAwards',

In [27]:
# I'll try by removing the gruop by
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (MAX(?numAwards) AS ?maxAwards)
WHERE {
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmName', 'Hannah and Her Sisters'), ('maxAwards', '3')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmName', 'The Hateful Eight'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q2907178'), ('filmName', 'Blue Jasmine'), ('maxAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('maxAwards', '1')]
[('d

In [30]:
# I'll try by filtering on the maxAwards
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (MAX(?numAwards) AS ?maxAwards)
WHERE {
{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
FILTER (?maxAwards=2)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [34]:
# I'll try the same path as the task 5: take the film which won more Academy Awards for the two directors separately, then do the union
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmName', 'Annie Hall'), ('numAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmName', 'Hannah and Her Sisters'), ('numAwards', '3')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q2907178'), ('filmName', 'Blue Jasmine'), ('numAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('numAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('numAwards', '1')]
[('director',

In [35]:
# i'll retrieve only the first result
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmName', 'Annie Hall'), ('numAwards', '4')]
1


In [36]:
# Now I'll add also the query for Tarantino
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName ?numAwards
WHERE{
{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 1
}
UNION
{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 1
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmName', 'Annie Hall'), ('numAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('numAwards', '2')]
2


Final query for this task

In [37]:
# write your final query
# My final result is without the film. I couldn't find a way to get the film, because of the gruop by clause.
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName ?numAwards
WHERE{
{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q25089)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 1
}
UNION
{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P166 ?academyAward.
   ?director sc:name ?directorName .
   ?film sc:name ?filmName .
   ?academyAward sc:name ?academyAwardName .
   FILTER(?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
ORDER BY DESC(?numAwards)
LIMIT 1
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmName', 'Annie Hall'), ('numAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('numAwards', '2')]
2


## Task 8
Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [14]:
# write your queries
# From previous query I have found P1411 nominated for, I'll use it
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P1411 ?academyAward.
   ?director sc:name ?directorName .
   ?academyAward sc:name ?academyAwardName
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q102427'), ('academyAwardName', 'Academy Award for Best Picture')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director')]
[('academyAward', 'http://www.wikidata.org/entity/Q106301'), ('academyAwardName', 'Academy Award for Best Supporting Actress')]
[('academyAward', 'http://www.wikidata.org/entity/Q281939'), ('academyAwardName', 'Academy Award for Best Film Editing')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('academyAward', 'http://www.wikidata.org/entity/Q103916'), ('academyAwardName', 'Academy Award for Best Actor')]
[('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('academyAward', 'http://www.wikidata.org/entity/Q131520'), ('academyAwardName', 'Academy Award for Best Cinematography')]
[('ac

In [15]:
# Now I'll group by on Academy Award
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName (COUNT(?film) AS ?nomination)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P1411 ?academyAward.
   ?director sc:name ?directorName .
   ?academyAward sc:name ?academyAwardName
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?academyAward ?academyAwardName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q102427'), ('academyAwardName', 'Academy Award for Best Picture'), ('nomination', '6')]
[('academyAward', 'http://www.wikidata.org/entity/Q488645'), ('academyAwardName', 'Academy Award for Best Sound Editing'), ('nomination', '2')]
[('academyAward', 'http://www.wikidata.org/entity/Q488651'), ('academyAwardName', 'Academy Award for Best Original Score'), ('nomination', '1')]
[('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor'), ('nomination', '7')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director'), ('nomination', '9')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay'), ('nomination', '19')]
[('academyAward', 'http://www.wikidata.org/entity/Q830079'), ('academyAwardName', 'Academy Award for Best Sound'), ('nom

Final query for this task

In [16]:
# write your final query
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName (COUNT(?film) AS ?nomination)
WHERE {
   # bind something
   ?academyAward wdt:P31 wd:Q19020 .
   ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?director;
        wdt:P1411 ?academyAward.
   ?director sc:name ?directorName .
   ?academyAward sc:name ?academyAwardName
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?academyAward ?academyAwardName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q102427'), ('academyAwardName', 'Academy Award for Best Picture'), ('nomination', '6')]
[('academyAward', 'http://www.wikidata.org/entity/Q488645'), ('academyAwardName', 'Academy Award for Best Sound Editing'), ('nomination', '2')]
[('academyAward', 'http://www.wikidata.org/entity/Q488651'), ('academyAwardName', 'Academy Award for Best Original Score'), ('nomination', '1')]
[('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor'), ('nomination', '7')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director'), ('nomination', '9')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay'), ('nomination', '19')]
[('academyAward', 'http://www.wikidata.org/entity/Q830079'), ('academyAwardName', 'Academy Award for Best Sound'), ('nom